<a href="https://colab.research.google.com/github/MartinVIllesca/CC6205-Procesamiento-de-Lenguaje-Natural/blob/master/minitarea2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Minitarea 2

Nombre: Martin Valderrama Illesca

---------------------------
## Language Models (3 pts)
Estas preguntas son teóricas y deben ser resueltas con desarrollo, pero sin código. Por favor usen $\LaTeX$ para las fórmulas. El material del curso y los videos deberian ser suficientes para que puedan responder.


### Pregunta 1 (1 pt)
Asuma que tiene un modelo de lenguaje de trigramas (simple) entrenado sobre un corpus C. Muestre cómo el modelo le asigna una probabilidad a la oración `el perro le ladra al gato` usando los parámetros estimados a partir del corpus  $q(w_i | w_{i-2}, w_{i-1})$ .

**Respuesta:**

La probabilidad es calculada de la siguiente forma:

$$p(\text{el perro le ladra al gato STOP}) = \prod_{i=1}^{6} q(w_i | w_{i-2}, w_{i-1}) $$

Con $w \in \{ \text{el}, \text{perro}, \text{le}, \text{ladra}, \text{al}, \text{gato}\}$ si es que no se hiciera un procesamiento del corpus antes, es decir, sacar las STOP Words o hacer stemming. En el caso de que se haga un procesamiento, se sigue el mismo procedimiento pero se reduce el corpus. Entregando, por ejemplo un corpus con los sgtes elementos, es decir $(w_i | w_{i-2}, w_{i-1}) \in \{ \{\text{perro | 1, 1}\}, \{\text{ladrar | 1, perro}\}, \{\text{gato | perro, ladrar}\}\}$.


$$p(\text{perro ladrar gato STOP}) = q(\text{perro} | *, *) \times q(\text{ladrar} | *, \text{perro}) \times q(\text{gato} | \text{perro}, \text{ladrar}) \times q(\text{STOP} | \text{ladrar}, \text{gato}) $$



### Pregunta 2 (1 pt)
Muestre cómo se calcularía  $q(w_{i} | w_{i-2}, w_{i-1})$  usando un modelo que interpola un modelo de lenguajes de trigramas, bigramas, y unigrama.

**Respuesta:**

Para calcular la probabilidad interpolada, se debe calcular por separado la probabilidad de cada unigrama, bigrama y trigrama, para luego generar la probabilidad del trigrama como una combinación lineal de las probabilidades del trigrama, bigrama y unigrama. Es decir,

Sea $q(w_i | w_{i-2}, w_{i-1})$, $q(w_i | w_{i-1})$ y $q(w_i)$ donde cada una se calcula como casos favorables sobre casos totales.

$$q(w_i | w_{i-2}, w_{i-1}) = \frac{\text{Count}(w_{i-2}, w_{i-1}, w_i)}{\text{Count} (w_{i-2}, w_{i-1})}$$

$$q(w_i | w_{i-1}) = \frac{\text{Count}(w_{i-1}, w_i)}{\text{Count} (w_{i-1})}$$

$$q(w_i) = \frac{\text{Count}(w_i)}{\text{Count} ()}$$

En el caso de $q(w_i | w_{i-2}, w_{i-1})$ se tiene que existen $\lambda_1, \lambda_2, \lambda_3$ tal que, $\sum \lambda_i = 1$:

$$q(w_i | w_{i-2}, w_{i-1}) = \lambda_1 \times q(w_i | w_{i-2}, w_{i-1}) + \lambda_2 \times q(w_i | w_{i-1}) + \lambda_3 \times q(w_i)$$

Se asume que los trigramas, bigramas o unigramas están bien definidos en el corpus, o se hace un ajuste de las probabilidades con métodos de descuento, para que las probabilidades no sean nulas.

### Pregunta 3 (1 pt)
¿Qué ventajas tiene el modelo interpolado sobre el modelo de trigramas simple?

**Respuesta:**

El modelo interpolado permite integrar modelos de gran tamaño y de menor tamaño. Los modelos de gran tamaño tienen menos sesgo pero más variabilidad, y los modelos pequeños (por ejemplo, unigramas) tienen menor variabilidad pero son muy sesgados.



-----------------------
## Naive Bayes (3 pts)
En esta parte de la minitarea deberan programar Naive Bayes Multinomial usando Laplace Smothing. Las referencias las pueden encontrar en el material del curso y los videos del profesor.

A continuacion se presentan un conjunto de documentos de training divididos en 2 categorias distintas. Ustedes deberan clasificar los documentos del test set usando Naive Bayes con Laplace Smothing.

Por favor, documenten su codigo. Escriban que hacen las funciones, que reciben, que entregan, etc. Si en algun lugar de su codigo hacen algo que creen que debe ser explicado, pongan comentarios, son bienvenidos. 


**Underflow prevention:** En vez de hacer muchas multiplicacions de `float`s, reemplacenlas por sumas de logaritmos para prevenir errores de precision. Revisen la diapo 69 de las slides. 

NOTA: Sobre las `namedtuple`s. Es el tipo de los documentos. Son objetos inmutable que tienen dos atributos: `words` donde estan las palabras del documento y `class_` donde se guarda la clase de ese documento. Estos objetos son inmutables, lo que quiere decir que si quieren modificar un documento y cambiarle la clase, tienen que crear otro documento. Otra cosa es que son tuplas como cualquier otra, es decir se pueden acceder usando indices como `doc[0]` o `doc[1]`. Son libres de implementar su solucion como quieran, si no les gusta este tipo de dato usen otro.



In [1]:
from collections import namedtuple
from pprint import pprint

document = namedtuple(
    "document", ("words", "class_")  # avoid python's keyword collision
)

train_set = (
    document(words=('w03', 'w01', 'w02', 'w06', 'w02', 'w08', 'w07'), class_=0),
    document(words=('w05', 'w04', 'w00', 'w06', 'w09', 'w07', 'w06', 'w09', 'w05'), class_=1),
    document(words=('w07', 'w06', 'w00', 'w08', 'w01', 'w08', 'w08', 'w09', 'w02'), class_=0),
    document(words=('w08', 'w09', 'w02', 'w06', 'w05', 'w08', 'w07'), class_=1),
    document(words=('w09', 'w08', 'w05', 'w08', 'w05', 'w00', 'w08'), class_=1),
    document(words=('w05', 'w05', 'w06', 'w01', 'w06', 'w08', 'w02'), class_=1),
    document(words=('w04', 'w03', 'w07', 'w05', 'w04', 'w00', 'w02'), class_=0),
    document(words=('w01', 'w00', 'w01', 'w04', 'w09', 'w02', 'w04', 'w07'), class_=1)
)
print("Train documents:")
pprint(train_set)


test_set = (document(words=('w02', 'w09', 'w06', 'w01', 'w05', 'w04', 'w03', 'w03'), class_=None),)
print("\nTest documents:")
pprint(test_set)

Train documents:
(document(words=('w03', 'w01', 'w02', 'w06', 'w02', 'w08', 'w07'), class_=0),
 document(words=('w05', 'w04', 'w00', 'w06', 'w09', 'w07', 'w06', 'w09', 'w05'), class_=1),
 document(words=('w07', 'w06', 'w00', 'w08', 'w01', 'w08', 'w08', 'w09', 'w02'), class_=0),
 document(words=('w08', 'w09', 'w02', 'w06', 'w05', 'w08', 'w07'), class_=1),
 document(words=('w09', 'w08', 'w05', 'w08', 'w05', 'w00', 'w08'), class_=1),
 document(words=('w05', 'w05', 'w06', 'w01', 'w06', 'w08', 'w02'), class_=1),
 document(words=('w04', 'w03', 'w07', 'w05', 'w04', 'w00', 'w02'), class_=0),
 document(words=('w01', 'w00', 'w01', 'w04', 'w09', 'w02', 'w04', 'w07'), class_=1))

Test documents:
(document(words=('w02', 'w09', 'w06', 'w01', 'w05', 'w04', 'w03', 'w03'), class_=None),)


In [37]:
# Tu respuesta

# para entrenar a Naive Bayes: Se quiere calcular la clase del documento dadas las palabras que tiene (test_set)

# se calculan la frecuencia de clases en el dataset de entrenamiento
c_0 = sum([d.class_ == 0 for d in train_set])
print(c_0, len(train_set))
c_1 = sum([d.class_ == 1 for d in train_set])
print(c_1, len(train_set))

# se calcula la frecuencia de las palabras en el dataset de entrenamiento
W_0 = {} # diccionario de palabras y su frecuencia
W_1 = {}
for d in train_set:
  if d.class_ == 0:
    for w in d.words:
      if w in W_0.keys(): W_0[w] += 1 # se agrega uno a la frecuencia de la palabra w
      else: W_0[w] = 1 # se cuenta por primera vez la palabra de la clase 0
  elif d.class_ == 1:
    for w in d.words:
      if w in W_1.keys(): W_1[w] += 1
      else: W_1[w] = 1

# print(W_0, sum(W_0.values()), len(W_0))
# print(W_1, sum(W_1.values()), len(W_1))


# calcular la probabilidad de que el documento sea de una clase dadas las palabras del documento
for d in test_set:
  prob_c0 = 0
  prob_c1 = 0

  for w in d.words:
    if not w in W_0.keys(): W_0[w] = 0
    if not w in W_1.keys(): W_1[w] = 0
    prob_c0 += (W_0[w] + 1) / (sum(W_0.values()) + len(W_0))
    prob_c1 += (W_1[w] + 1) / (sum(W_1.values()) + len(W_1))

  print('Prob de C0: {:.2f}'.format(prob_c0 * c_0 / len(train_set)))
  print('Prob de C1: {:.2f}'.format(prob_c1 * c_1 / len(train_set)))


3 8
5 8
Prob de C0: 0.27
Prob de C1: 0.45
